In [1]:
import scipy.io
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D, MaxPool2D, Flatten
from keras import optimizers
import keras.utils as np_utils
from keras import backend as K
import matplotlib.pyplot as plt
import sys
import urllib
import os
import tensorflow_probability as tfp

Using TensorFlow backend.


In [0]:
####### download mnist file from website 
if not os.path.isfile("train-images-idx3-ubyte.gz"):
    urllib.request.urlretrieve("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz","train-images-idx3-ubyte.gz")
if not os.path.isfile("train-labels-idx1-ubyte.gz"):
    urllib.request.urlretrieve("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz","train-labels-idx1-ubyte.gz")

######## reading from the .gz files
import gzip
f=gzip.open("train-images-idx3-ubyte.gz",'rb')
buf=f.read(4)
magic_num=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))
if magic_num != 2051:
    print("invalid file ... exiting...")
    exit()
buf=f.read(4)
N=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))[0]
buf=f.read(4)
H=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))[0]
buf=f.read(4)
W=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))[0]
train_img=np.frombuffer(f.read(H*W*N),dtype=np.uint8)
train_img=train_img.reshape(N,H,W,1)
f.close()

f=gzip.open("train-labels-idx1-ubyte.gz",'rb')
buf=f.read(4)
magic_num=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))
if magic_num != 2049:
    print("invalid file ... exiting...")
    exit()
buf=f.read(4)
N=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))[0]
train_label=np.frombuffer(f.read(N),dtype=np.uint8)
train_label=train_label.reshape(N,1)
f.close()
train_label=np_utils.to_categorical(train_label);

In [3]:
if K.image_data_format() == 'channels_last':
    train_img = train_img.reshape(train_img.shape[0],train_img.shape[1], train_img.shape[2], train_img.shape[3])
else:
    train_img = train_img.reshape(train_img.shape[0], 1, train_img.shape[1], train_img.shape[2])
print(train_img.shape)

(60000, 28, 28, 1)


In [0]:
def new_model(m,n,o,b):
    model=Sequential()
    model.add(Conv2D(filters=5,kernel_size=(5,5),padding='Valid',activation='relu',input_shape=(28,28,1)));
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Conv2D(filters=5,kernel_size=(5,5),padding='Valid',activation='relu'));
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Flatten())
    if b==1:
        model.add(Dense(n,activation='sigmoid',use_bias=True))
    else:
        model.add(Dense(n,activation='sigmoid',use_bias=False))
    model.add(Dense(o[1],activation='softmax',use_bias=False))
    return model

def get_input(model,train_img,n):
    get_layer=K.function([model.layers[0].input],[model.layers[n].output])
    t_img=get_layer([train_img])[0]
    return t_img

def random_weight(a):
    c=a.copy
    c=-1+np.random.random(a.shape)*2
    return c

def list_sum_vec(c,b):
    a=c.copy()
    for i in range(len(a)):
        a[i]=a[i]+b
    return a

def list_mul_vec(c,b):
    a=c.copy()
    for i in range(len(a)):
        a[i]=a[i]*b
    return a

def list_sub_vec(c,b):
    a=c.copy()
    for i in range(len(a)):
        a[i]=a[i]-b
    return a

def list_div_vec(c,b):
    a=c.copy()
    for i in range(len(a)):
        a[i]=a[i]/b
    return a

def list_sum_list(c,b):
    a=c.copy()
    for i in range(len(a)):
        a[i]=a[i]+b[i]
    return a
def list_mul_list(c,b):
    a=c.copy()
    for i in range(len(a)):
        a[i]=np.multiply(a[i],b[i])
    return a
def list_sub_list(c,b):
    a=c.copy()
    for i in range(len(a)):
        a[i]=a[i]-b[i]
    return a

def list_div_list(c,b):
    a=c.copy()
    for i in range(len(a)):
        a[i]=np.divide(a[i],b[i])
    return a

def list_rand(c):
    a=c.copy()
    for i in range(len(a)):
        a[i]=0.01*(np.random.random(a[i].shape))
    return a

def cross_list(a,b,p):
    c1=a.copy()
    c2=a.copy()
    for i in range(len(a)):
        c1[i]=(a[i]*p)+(b[i]*(1-p))
        c2[i]=(a[i]*(1-p))+(b[i]*p)
    return c1,c2

def list_mutation(c,b):
    a=c.copy()
    for i in range(len(a)):
        a[i]=a[i]+(-1+np.random.random(a[i].shape)*2)*b
    return a

In [0]:
use_bias=1

model=new_model(train_img.shape,300,train_label.shape,use_bias)
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_img,train_label,epochs=0,batch_size=100)
model.summary()
l=model.get_weights()

pop_sz=10
pop=[]
vel=[]
obj=[]
for i in range(pop_sz):
    for j in range(len(l)):
        l[j]=random_weight(l[j])
    pop.append(l.copy())
    for j in range(len(l)):
        l[j]=random_weight(l[j])
    vel.append(l.copy())
for i in range(pop_sz):
    l=pop[i]
    model.set_weights(l)
    t_img=get_input(model,train_img,len(model.layers)-2)
    t_img=tf.Session().run(tf.matmul(tfp.math.pinv(t_img),train_label))
    l[len(l)-1]=t_img
    model.set_weights(l)
    v=model.evaluate(train_img,train_label,batch_size=1000)
    pop[i]=l.copy()
    obj.append(v[0].copy())
# minimizing the loss
print(obj)
l_pop=pop.copy()
l_obj=obj.copy()
g_pop=pop[np.argmin(obj)].copy()
g_obj=obj[np.argmin(obj)].copy()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 24, 24, 5)         130       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 12, 12, 5)         0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 8, 8, 5)           630       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 4, 4, 5)           0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 80)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 300)               24300     
_________________________________________________________________
dense_18 (Dense)             (None, 10)                3000      
Total para

In [33]:
momentum=0.008
g_vel=0.2
l_vel=0.008
iter_c=0
max_iter=100
plot_local_obj_mean=[]
plot_g_obj=[]

while iter_c<max_iter:
    for i in range(pop_sz):
        tp=list_mul_list(list_mul_vec(list_rand(pop[i]),g_vel),list_sub_list(g_pop,pop[i]))
        tpp=list_mul_list(list_mul_vec(list_rand(pop[i]),l_vel),list_sub_list(l_pop[i],pop[i]))
        vel[i]=list_sum_list(list_mul_vec(vel[i],momentum),list_sum_list(tp,tpp))
        npop=list_sum_list(pop[i],vel[i])
        c1,c2=cross_list(npop,g_pop,0.2)
        model.set_weights(c1)
        v1=model.evaluate(train_img,train_label,batch_size=1000,verbose=0)
        model.set_weights(c2)
        v2=model.evaluate(train_img,train_label,batch_size=1000,verbose=0)
        if v1[0]<obj[i]:
            pop[i]=c1.copy()
            obj[i]=v1[0].copy()
        if v2[0]<obj[i]:
            pop[i]=c2.copy()
            obj[i]=v2[0].copy()
        if obj[i]<g_obj:
            g_pop=pop[i].copy()
            g_obj=obj[i].copy()
        if obj[i]<l_obj[i]:
            l_pop[i]=pop[i].copy()
            l_obj[i]=obj[i].copy()
            
    plot_local_obj_mean.append(np.mean(l_obj))
    plot_g_obj.append(g_obj)
    print(iter_c," : ",g_obj," : ",np.mean(l_obj)," : ",np.mean(obj))
    iter_c=iter_c+1

0  :  1.9280757129192352  :  1.9335993889967604  :  1.933599389592807
1  :  1.9280757129192352  :  1.9291007202863693  :  1.9291007208824158
2  :  1.9280757129192352  :  1.9282797396183013  :  1.9282797402143477
3  :  1.9280757129192352  :  1.9281142606337867  :  1.9281142612298332
4  :  1.9280757129192352  :  1.9280836137135826  :  1.9280836143096287
5  :  1.9280757129192352  :  1.9280773735046388  :  1.9280773741006851
6  :  1.9280757129192352  :  1.9280760620037714  :  1.9280760625998181


IndexError: ignored

In [31]:
print(obj)

[1.956766563653946, 1.9300814628601075, 1.9304521640141805, 2.090738912423452, 1.9319131513436636, 1.9280757188796998, 1.9364234983921051, 2.0819326202074686, 1.9307277123133342, 1.9290718952814738]
